In [2]:
import pandas as pd
import csv

In [ ]:
#MERGE query data to event data

In [2]:
df_panama_papers = pd.read_table('data/final/query_panama_papers_data.txt', sep='\t', encoding='utf-8', header=0)
df_offshore = pd.read_table('data/final/query_offshore_data.txt', sep='\t', encoding='utf-8', header=0)
df_leak = pd.read_table('data/final/query_leak_data.txt', sep='\t', encoding='utf-8', header=0)
df_fonseca = pd.read_table('data/final/query_fonseca_data.txt', sep='\t', encoding='utf-8', header=0)
df_tax_haven = pd.read_table('data/final/query_tax_haven_data.txt', sep='\t', encoding='utf-8', header=0)

In [3]:
df_panama_papers['query'] = 'panama papers'
df_offshore['query'] = 'offshore'
df_leak['query'] = 'leak'
df_fonseca['query'] = 'fonseca'
df_tax_haven['query'] = 'tax haven'

In [4]:
df = pd.concat([df_panama_papers, df_offshore, df_leak, df_fonseca, df_tax_haven])
df.shape

(2844241, 21)

In [5]:
#handle event overlap between queries
#not query specific
df = df.drop_duplicates('id_str')
df.shape

(2346998, 21)

In [6]:
#convert date format to python datetime
df['created_at'] = pd.to_datetime(df['created_at'])

In [7]:
#sort data on date posted ascending
df = df.sort_values(by='created_at', ascending=True)

In [13]:
#slice data on event dates
df = df[(df['created_at'] >= '2016-04-03 00:00:00') & (df['created_at'] <= '2016-04-14 00:00:00')]
df.shape

(2295892, 21)

In [ ]:
#DATA BASE FILE

In [22]:
#create, add index as master id, rename twitter id
df.reset_index(drop=True, inplace=True)
df = df.reset_index().rename(columns={'index' : 'master_id', 'id_str' : 'twitter_id'})

In [28]:
df.to_csv('data/final/event_panama_papers_data.txt', sep='\t', encoding='utf-8', header=True, index=False)

In [ ]:
#MASTER FILE

In [4]:
df_master = df[['master_id', 'twitter_id', 'created_at', 'text', 'is_retweet', 'tweet_type']]

In [5]:
df_master['event'] = 'Panama Papers'
df_master['event_description'] = 'The Panama Papers are a set of 11.5 million confidential documents detailing information about more than 214,000 offshore companies compiled by the Panamanian corporate service provider Mossack Fonseca, including the identities of company shareholders and directors.'

c:\miniconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
c:\miniconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [31]:
#save id, text as master, with index
df_master.to_csv('data/final/event_panama_papers_master.txt', sep='\t', encoding='utf-8', header=True, index=False)

In [ ]:
#save sample for annotation
#no retweets, only text
#sample 1K
smpl = df_master[(df_master['is_retweet'] == False)].sample(100000)
#format for crowdflower import
#delimited by ;
#double quote strings
smpl.to_csv('data/final/event_panama_papers_master_sample.csv', sep=',', quoting=csv.QUOTE_NONNUMERIC, encoding='utf-8', header=True, index=False)